In [3]:
# import libraries
import cv2
import numpy as np

# init variables
markerIDs = [23, 25, 30, 33]
scaling_fac_x = .008 
scaling_fac_y = .012

# init the camera
cap = cv2.VideoCapture(0)
# importing the video and preloading it
vid = cv2.VideoCapture("testing.mp4")
retVid, frameVid = vid.read()
ptsSrc = [[0, 0], [frameVid.shape[1], 0], [frameVid.shape[1], frameVid.shape[0]], [0, frameVid.shape[0]]]
ptsSrcm = np.asarray(ptsSrc)

# Load the ArUco dictionary.
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

while True:
    # Capture frame-by-frame
    retCap, frame = cap.read()
    
    # Detect the ArUco markers in the frame.
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, arucoDict)
    # If all four ArUco markers are detected, start importing the video and applying the homography.
    if ids is not None and len(ids) == 4:
        # Extract the reference points for the ROI.
        ref = []
        for i in range(len(ids)):
            index = np.squeeze(np.where(ids == markerIDs[i]))
            ref.append(np.squeeze(corners[index[0]])[i])
            
        # Calculate the scaling factors for the destination points.
        x_distance = np.linalg.norm(ref[0] - ref[1])
        y_distance = np.linalg.norm(ref[0] - ref[2])

        delta_x = round(scaling_fac_x * x_distance)
        delta_y = round(scaling_fac_y * y_distance)

        # Apply the scaling factors to the ArUco Marker reference points to make.
        # the final adjustment for the destination points.
        pts_dst = [[ref[0][0] - delta_x, ref[0][1] - delta_y]]
        pts_dst = pts_dst + [[ref[1][0] + delta_x, ref[1][1] - delta_y]]
        pts_dst = pts_dst + [[ref[2][0] + delta_x, ref[2][1] + delta_y]]
        pts_dst = pts_dst + [[ref[3][0] - delta_x, ref[3][1] + delta_y]]
        
        pts_src_m = np.asarray(ptsSrc)
        pts_dst_m = np.asarray(pts_dst)

        retVid, frameVid = vid.read()
        if not retVid:
            vid = cv2.VideoCapture("testing.mp4")
            retVid, frameVid = vid.read()
        
        pts_dst_m = np.asarray(pts_dst)

        # Calculate the homography.
        h, mask = cv2.findHomography(pts_src_m, pts_dst_m, cv2.RANSAC)

        # Warp source image onto the destination image.
        warped_image = cv2.warpPerspective(frameVid, h, (frameVid.shape[1], frameVid.shape[0]))

        # Create a mask for the destination image.
        mask = np.zeros((frameVid.shape[0], frameVid.shape[1]), dtype=np.uint8)

        cv2.fillConvexPoly(mask, pts_dst_m.astype(int), (255, 255, 255), cv2.LINE_AA)

        # Copy the mask into 3 channels.
        warped_image = warped_image.astype(float)
        mask3 = np.zeros_like(warped_image)
        for i in range(0, 3):
            mask3[:, :, i] = mask / 255

        # Create inverse mask.
        mask3_inv = 1 - mask3

        # Create black region in destination frame ROI.
        frame_masked = cv2.multiply(frame.astype(float), mask3_inv)

        # Create final result by adding the warped image with the masked destination frame.
        frame_out = cv2.add(warped_image, frame_masked)
        frame_out = frame_out.astype(np.uint8)
        cv2.imshow("frame", frame_out)
    else:
        frameDetected = frame.copy()
        cv2.aruco.drawDetectedMarkers(frameDetected, corners, ids)
        cv2.imshow("frame", frameDetected)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows() 
        break




